This notebook measures the intensities and errors of CO 2-1 emission at the locations of all of the SNe in our sample across all of the resolutions in the survey. This is used to prepare to answer the question in section 3.1 Do we see gas at supernova locations? 

We will use this to uncover what resolution will be best for this study.

In [1]:
# IMPORT PACKAGES

#global
import numpy as np
from matplotlib import pyplot as plt
import astropy
import astropy.io.fits as pyfits
from astropy.table import Table, join
from astropy.wcs import WCS
from astropy.io import ascii
from astropy.io import fits
import os
from reproject import reproject_interp

#local
import sys
sys.path.append('/home/mayker.1/Desktop/PythonFunctions')

from findResolution import findRes
from getMapValue    import getValue
from nonZeroError   import findErrVals
from nonZeroError   import nonZeroErrArray
from nonZeroError   import findSignal
from findPercentiles import findStats

In [2]:
# DEFINE FUNCTIONS

def genFileName(galaxy, mapType, res, telOrient):
    
    """
    Generates the filenames for the CO maps.
    
    Parameters
    ----------
    galaxy    : string : name of galaxy
    mapType   : string : "int", "err", or "EW"   
    res       : string : resolution of map ("" (native), "_60pc", "_90pc", "_120pc", "_150pc", etc.)
    telOrient : string : 12m+7m+tp, 7m+tp, etc.
    
    Returns
    -------
    fileName : string : full path to file on Tycho.
        (/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc2997/ngc2997_12m+7m+tp_co21_150pc_broad_mom0.fits)

    """
    
    if mapType == "int":
        mapTypeStr = "_broad_mom0"
    elif mapType == "err":
        mapTypeStr = "_broad_emom0"
    elif mapType  == "EW":
        mapTypeStr = "_strict_ew"
    else:
        print("Wrong Map Type.")
        
    preamble = "/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/" 
    
    fileName = preamble + galaxy + "/" + galaxy + "_" + telOrient + "_co21" + res + mapTypeStr + ".fits"
    return(fileName)


def getAlphaCO(acofileName, intfileName, SNra, SNdec):
    """
    Pulls the metallicity dependent Alpha CO value at the SN location using Sun+2020 maps
    or assigns a value of alpha CO = 4.35/0.65 if no Sun value is available.
    """

    if(os.path.isfile(acofileName) == True) and (os.path.isfile(intfileName) == True):
        hdu_int = pyfits.open(intfileName)
        hdu_aco = pyfits.open(acofileName)
        acoMap, footprint = reproject_interp(hdu_aco, hdu_int[0].header)

        new_hdu = fits.PrimaryHDU(acoMap, hdu_int[0].header)
        outco = "../Data/ACOrewrites/temp.fits"
        new_hdu.writeto(outco, overwrite=True)
        
        alphaCO = getValue(outco,float(SNra),float(SNdec))
        if np.isfinite(alphaCO) == False:
            print(intfileName)
            alphaCO = 6.7
    else:
        print("NO ACO file: ", acofileName)
        alphaCO = 4.35/0.65
    
    return(alphaCO)

def findSNRErr(errorArray, i, res):
    """
    Finds the lowest non-zero error for use in calculating the SNR in the map if the pixel has a non-real error associated with it.
    Otherwise just uses the pixel error.
    """
    
    if(errorArray[i] <= 0.0):   
        hdulist = pyfits.open(genFileName(galaxies[i], "err", res, telOrient[i]))
        map = hdulist[0].data 
        flatMap = map.flatten()
        flatMap = flatMap[flatMap > 0.0]
        flatMap.sort()
        usedErr = flatMap[0]
    else: 
        usedErr = errorArray[i]
    
    return(usedErr)
    
def percentiles(array):

    vals = np.asarray(array)
    
    for i in range(len(vals)):
        if vals[i] < 0.0:
            vals[i] = 0.0
    
    vals = vals[np.isfinite(vals)]
    
    percentile = np.linspace(0,1,len(vals))
    stats = findStats(percentile, vals)

    return(stats, len(vals))

In [3]:
# import SNeCO data

dataFile = '../Data/3.SNe+GalData.csv'
dataTable = Table.read(dataFile, format='csv') 
dataTable.colnames
#dataTable['AlphaCOFile']


['galaxy',
 'SN_name',
 'SN_type',
 'SN_ra',
 'SN_dec',
 'SN_xCoord',
 'SN_yCoord',
 'SN_co21int',
 'SN_co21err',
 'SN_co21ew',
 'map_beamsize',
 'orient_ra',
 'orient_dec',
 'dist',
 'orient_posang',
 'orient_incl',
 'size_r25',
 'telOrient',
 'ImageFile150',
 'ErrFile150',
 'AlphaCOFile']

In [4]:
# pull galaxy names and telescope orientation to generate the file lists

galaxies  = dataTable['galaxy']
telOrient = dataTable['telOrient']
SNname    = dataTable['SN_name']
SNra      = dataTable['SN_ra']
SNdec     = dataTable['SN_dec']
beamsize  = dataTable['map_beamsize']
telOrient = dataTable['telOrient']
distance  = dataTable['dist']
acoFile   = dataTable['AlphaCOFile']

In [13]:
# set up resolution array and galaxies without repeats
res = ['60', '90', '120', '150', '500', '750', '1000']
gals = list(dict.fromkeys(galaxies))
print(gals)

# find percentiles of all pixels from galaxy maps
for i in range(len(res)):
    
    resIntPixArr = []

    resStr = "_" + res[i] +  "pc"

    for j in range(len(gals)):
                
        galIntPixArr = []
        
        IntFileName = genFileName(gals[j], "int", resStr, telOrient[j])
        
        if(os.path.isfile(IntFileName) == True):
            
            hdulist = pyfits.open(IntFileName)
            map = hdulist[0].data
            flatmap=map.flatten()

            #remove nans
            keep  = np.where(np.isfinite(flatmap))
            galIntPixArr = flatmap[keep]
            
            #add to full list
            resIntPixArr = np.concatenate((galIntPixArr, resIntPixArr), axis=0)
            #print("file for ", gals[j], " at resolution ", res[i])
            
        else:
            pass
            #print("No file for ", galaxies[j], " at resolution ", res[i])
        
    resArr = np.sort(resIntPixArr)
    resCS = np.cumsum(resArr)/np.sum(resArr)
    perArr = np.interp([0.16, 0.5, 0.84],resCS,resArr)

    print("Resolution: ", res[i], " Percentiles: ", perArr)
            


['circinus', 'ngc0253', 'ngc0628', 'ngc1068', 'ngc1087', 'ngc1097', 'ngc1365', 'ngc1433', 'ngc1511', 'ngc1559', 'ngc1566', 'ngc1637', 'ngc1672', 'ngc2775', 'ngc2997', 'ngc3239', 'ngc3627', 'ngc4254', 'ngc4303', 'ngc4321', 'ngc4424', 'ngc4457', 'ngc4496a', 'ngc4536', 'ngc4579', 'ngc4945', 'ngc5128', 'ngc5236', 'ngc5530', 'ngc6744', 'ngc7793']
Resolution:  60  Percentiles:  [ 3.36026913  9.58205602 50.29678407]
Resolution:  90  Percentiles:  [ 2.95562233 10.00328541 65.07420286]
Resolution:  120  Percentiles:  [  3.16950127  14.07367865 179.10026843]
Resolution:  150  Percentiles:  [  3.02471822  13.75703453 172.18191637]
Resolution:  500  Percentiles:  [  2.30941766   9.56774433 110.82564537]
Resolution:  750  Percentiles:  [ 2.23616652  8.60419375 86.35067781]


ValueError: array of sample points is empty